Leetcode： 当问题中有排序数组时候，常用到binary search

           如果时间复杂度超过nlogn, 可以先排序再binary search
           
Binary Search，可以分成离散和连续两类。离散就是一个排好序的数组里面找数，连续就是在[0,x]这个连续区间内找一个符合条件的解。因为0–x是自然连续增加的，所以当然可以用Binary Search。

#### 离散Binary Search 通用模板：

    离散Binary Search用前闭后开区间[a,b)和前闭后闭区间[a,b]都可以。


前闭后开是一种STL风格的写法。这种写法的好处是：

    A 给遍历元素时，循环的结束时机提供一个简单的判断依据。只要尚未到达end()，循环就可以继续下去。写法要简单很多。

    B 不必对空区间采取特殊处理手段。空区间的begin()就等于end()。



我们可以跟后面的前闭后闭一比较就可以看出来。
        

In [6]:
def binarySearch_close_open(A, target): # A is sorted # 前闭后开 >= start  > start
    if not A:
        return -1
    start, end = 0, len(A)-1
    while start < end: # < not <=
        mid = start + (end - start)//2
        if A[mid] < target:
            start = mid + 1
        elif A[mid] > target:
            end = mid
        else:
            return mid
    return -1

A = [1, 2, 3, 4, 5]
target = 4
print(binarySearch_close_open(A, target))

3


注意：

1） 如果有重复元素的话，该模板返回该重复元素的任意位置(不一定是第一个，也不一定是最后一个）。

2） 需要特别注意的是，这种前闭后开区间的写法只能用于一旦找到就马上return pos的情况，而不能用于寻找重复元素出现的第一次或最后一次出现位置的情况，否则改动不当容易导致死循环！这种情况下应该用下面的前闭后闭区间模板。


In [7]:
def binarySearch_close_close_recursive(A, target): # A is sorted # 前闭后开 >= start  > start
    if not A:
        return -1
    def binarySearch_helper(start, end, target):
        if start > end:
            return -1
        mid = start + (end - start)//2
        if A[mid] == target:
            return mid
        if A[mid] < target:
            return binarySearch_helper(mid + 1, end, target)
        else:
            return binarySearch_helper(start, mid - 1, target)
    
    return binarySearch_helper(0, len(A)-1, target)

注意我们不需要写if (start == end)的情况，因为如果start == end, 后面的mid = start， 再call
binarySearch(a, mid + 1, end, target)和binarySearch(a, start, mid - 1, target) 都会出现(start > end) 的情况


In [8]:
def binarySearch_close_close(A, target): # A is sorted # 前闭后开 >= start  > start
    if not A:
        return -1
    start, end = 0, len(A)-1
    while start+1 < end: # < not <=
        mid = start + (end - start)//2
        if A[mid] < target:
            start = mid 
        elif A[mid] > target:
            end = mid
        else: # ==
            return mid
    if A[start] == target:
        return start
    if A[end] == target:
         return end
    return -1

这里有两个主要问题：

Q1: 为什么要用 start + 1 < end？而不是 start < end 或者 start <= end？

这是为了避免死循环。二分法的模板中分为两个部分：

通过 while 循环，将区间范围从 n 缩小到 2 （只有 start 和 end 两个点）。2)在 start 和 end 中判断是否有解。
如果我们用start < end 或者 start <= end, 在寻找目标最后一次出现的位置的时候，会出现死循环。

Q2: 为什么明明可以 start = mid + 1 偏偏要写成 start = mid?
A: 大部分时候，mid 是可以 +1 和 -1 的。在一些特殊情况下，比如寻找目标的最后一次出现的位置时，当 target 与 nums[mid] 相等的时候，是不能够使用 mid + 1 或者 mid - 1 的。因为会导致漏掉解。统一写成 start = mid / end = mid 并不会造成任何解的丢失，并且也不会损失效率——log(n) 和 log(n+1) 没有区别。

注意：

1） 如果有重复元素的话，该模板返回该重复元素的某个位置(不一定是第一个，也不一定是最后一个）。

2）当需要寻找重复元素出现的最后一个位置时，把return mid改为start = mid。最后两行顺序为

    if A[end] == target: return end

    if A[start] == target: return start

3）当需要寻找重复元素出现的第一个位置时，把return mid改为end = mid。最后两行顺序为

    if A[start] == target: return start

    if A[end] == target: return end

4) 当需要寻找刚好小于或等于target的第一个元素时，我的理解是while()后的情况可能是start<=target<=end，也有可能是target<start<end，或start<end<target。所以 # 注意次序！！！

    if (nums[start] == target) return start    #  nums[start] == target
    
    if (nums[end] == target) return end   #  nums[end] = target
    
    if (nums[end] < target) return end     #  nums[start]<nums[end]<target  
    
    if (nums[start] < target) return start    #  nums[start]<target<nums[end]
    
5) 当需要寻找刚好大于等于target的第一个元素

    if (nums[start] == target) return start
    
    if (nums[end] == target) return end
    
    if (nums[start] > target) return start  #  target < nums[start] < nums[end]
    
    if (nums[end] > target) return end   #  nums[start] < target < nums[end] 



连续Binary Search版本。跟离散版本差不多，而且还简单些。但是要注意找解的时候不能用==，而必须用一个很小的threshold来比较。小于这个threshold就可以认为相等. 下面是我写的一个double版的x/y的function(不用到/)。这里我认为double的被除数不会为0。欢迎指正。
代码如下：

In [9]:
def divide(x, y): # x, y are integers
    epsilon = 10**(-8)
    if abs(y) < epsilon:
        return float('inf') if y > 0 else float('-inf')
    is_negative = (x < 0)^(y < 0)
    x, y = abs(x), abs(y)
    start, end = 0, x
    while start < end:
        mid = start + (end - start)//2
        if abs(mid*y - x) < epsilon:
            return is_negative*mid
        elif mid*y - x < 0:
            start = mid
        else:
            end = mid
    

#### 九章算法二分法模板(四个要素)

①start+1<end

②start+(end-start)/2

③A[mid]==,<,>

④A[start] A[end] ? target 

二分的位置： 去掉无解的一半，保留有解的一半



一些关于Binary Search的要点：

Binary Search和Hash Table的比较：
我们都知道Binary Search的复杂度是O(logn)， 而Hash Table的复杂度是O(1)。但Hash Table有个问题是它的空间复杂度较大，而且Hash Table必须常驻内存，不能放在磁盘里面。
而Binary Search的数据不需要常驻内存，可以放在磁盘里面再读到内存里面。所以Binary Search不受内存限制。文件系统通常都用Binary Search



#### 题目类型

一般会给你一个数组，让你找数组中第一个/最后一个满足某个条件的位置



#### 相关题目

Sqrt(x) :last number that number^2<=x

Search a 2D Matrix :last row that matrix[row][0]<=target

search insert position :first positon>=target  OR  (last position<target)+1

count of smaller number

search for a range/number of target

search in a big/infinite sorted array

first bad version  :first version that is bad version

 find minimum in rotated sorted array:position<=last number

search in rotated sorted array

find peak element

recover rotated sorted array

rotate string

#### 其他一些问题

1 取中位数索引
mid = (left + right)//2 在某些计算机语言中可能会导致溢出

办法一：  left + (right - left)//2较优，虽然依然有可能导致溢出，比如 left 为负值时，但这种情况或者错误较少

办法二： 用无符号右移 （left + right）>>> 1 这种对于例如java适用，对于python而言，因为没有数值长短的问题，所以可以直接右移一位


#### 例题

Find Minimum in Rotated Sorted Array，

这题只需要讨论切下去那一刀的位置，相对于旋转点的关系即可。而这个位置可以通过这个旋转后的array 起始点和重点的位置判断。 以为需要注意的情况，是如果序列没有旋转，以及倒序的情况。这两种情况，可在尾部进行判断

In [8]:
def findMin(self, nums):
   
    if not nums:
        return None
    if len(nums) == 1:
        return nums[0]
    start, end = 0, len(nums)-1
    start_value, end_value = nums[0], nums[end]
    
    while(start + 1 < end):
        mid = start + (end - start) // 2
        if (nums[mid]>= start_value): 
            start = mid
        elif(nums[mid] <= end_value):
            end = mid
    result = 0         
    if (nums[start] < nums[end]):
        result = nums[start]
    else:
        result = nums[end]
    result = min(min(result,start_value ), end_value) # corner case!!!
    return result

Search in Rotated Sorted Array
这个问题相对要复杂些，因为不是找最小值，而是找一个具体的目标值，因此要分两大类，共四小类分别讨论。两大类是指target在旋转点左边，和在右边的情况。以及在这两种情况下切分点在target左边和右边的情况

In [9]:
# assumption: just find one index if there are duplicates
def search(A, target):
    if not A:
        return -1
    start, end = 0, len(A) - 1
    start_value, end_value = A[start], A[end]
    while(start + 1 < end):
        mid = start + (end - start) // 2
        if (target >= start_value):
            if (A[mid] >= start_value and A[mid] <= target):
                start = mid
            else:
                end = mid
        else:
            if (A[mid] >= target and A[mid] <= end_value):
                end = mid
            else:
                start = mid
    if (A[start] == target):
        return start
    if (A[end] == target):
        return end
    return -1

# general senario: find all indices of the given target
def search(nums, target):
    
    def helper(nums, target, start, mid, end):
        result = []
        if not nums[mid] == target:
            return []
        else:
            idx = mid
            while idx >= start and nums[idx] == target:
                result.append(idx) 
                idx -= 1
            idx = mid + 1
            while idx <= end and nums[idx] == target:
                result.append(idx)
                idx += 1
        return result
    
    if not nums:
        return -1
    start, end = 0, len(nums) - 1
    start_value, end_value = nums[0], nums[end]
    result = []
    if start_value == end_value and start_value == target:   
        result += helper(nums, target, start, start, end)
        result += helper(nums, target, start, end, end)
    else:
        while start + 1 < end:
            mid = start + (end - start)//2
            if target >= start_value:
                if nums[mid] >= start_value and nums[mid] < target:
                    start = mid
                elif nums[mid] == target:
                    return helper(nums, target, start, mid, end)
                else:
                    end = mid
            else:
                if nums[mid] <= end_value and nums[mid] > target:
                    end = mid
                elif nums[mid] == target:
                    return helper(nums, target, start, mid, end)
                else:
                    start = mid
    return result if result else -1
                
nums = [2, 2, 3, 4, 6, 1, 1, 2]
target = 2
print(search(nums, target))

[0, 1, 7]


Find Peak Element II 390

Question

There is an integer matrix which has the following features:
The numbers in adjacent positions are different.
The matrix has n rows and m columns.

For all i < m, A[0][i] < A[1][i] && A[n - 2][i] > A[n - 1][i].

For all j < n, A[j][0] < A[j][1] && A[j][m - 2] > A[j][m - 1].

We define a position P is a peek if:
A[j][i] > A[j+1][i] && A[j][i] > A[j-1][i] && A[j][i] > A[j][i+1] && A[j][i] > A[j][i-1]

Find a peak element in this matrix. Return the index of the peak.

Example
Given a matrix:
[ [1 ,2 ,3 ,6 ,5], [16,41,23,22,6], [15,17,24,21,7], [14,18,19,20,10], [13,14,11,10,9] ]

return index of 41 (which is [1,1]) or index of 24 (which is [2,2])

Challenge
Solve it in O(n+m) time.
If you come up with an algorithm that you thought it is O(n log m) or O(m log n), can you prove it is actually O(n+m) or propose a similar but O(n+m) algorithm?

In [ ]:
def find_peak(nums):
    def find_peak_helper(nums, idx):
        if not nums:
            return -1
        if len(nums) < idx:
            return -1
        start, end = 0, len(nums[idx]) - 1
        while start + 1 < end:
            mid = start + (end - start)//2
            if nums[idx][mid] > nums[idx][mid - 1] and nums[idx][mid] > nums[idx][mid + 1]:
                return mid
            elif nums[idx][mid] < nums[idx][mid - 1]:
                end = mid
            else:
                start = mid
                
        if nums[start] < nums[end]:
            return end
        else:
            return start
        
    if not nums:
        return None
    if len(nums) < 3 or len(nums[0]) < 3:
        return None
    start, end = 1, len(nums) - 2
    while start <= end:
        mid = start + (end - start)//2
        idx = find_peak_helper(nums, mid)
        if nums[mid][idx] > nums[idx - 1] and nums[mid][idx] > nums[mid][idx + 1]:
            return (mid, idx)
        elif nums[mid][idx] < nums[idx - 1]:
            end = mid - 1
        else:
            start = mid + 1
            
     

Wood Cut

Problem

Given n pieces of wood with length L[i] (integer array). Cut them into smallpieces to guarantee you could have equal or more than k pieces with the samelength. What is the longest length you can get from the n pieces of wood?Given L & k, return the maximum length of the small pieces.

Example

For L=[232, 124, 456], k=7, return 114.

Note
You couldn't cut wood into float length.

In [10]:
def woodCut(L, k):
    if sum(L) < k:
        return 0

    max_len = max(L)
    start, end = 1, max_len
    while start + 1 < end:
        mid = start + (end - start) / 2
        pieces_sum = sum([len_i / mid for len_i in L])
        if pieces_sum < k:
            end = mid
        else:
            start = mid

    # corner case
    if end == 2 and sum([len_i / 2 for len_i in L]) >= k:
        return 2

    return start

#### 扫描线法
二分法目的是找到某个元素或者是指标，能不能用二分法是看问题的解存在的集合能不能被分为更小的解存在集合和解不存在集合

如果不能划分，那么就不可以用二分法

下面介绍的扫描线法就是一个例子

Number of Airplanes in the Sky
Given an interval list which are flying and landing time of the flight. How many airplanes are on the sky at most?

Notice

If landing and flying happens at the same time, we consider landing should happen at first.

Have you met this question in a real interview? Yes
Example
For interval list

[
[1,10],
[2,3],
[5,8],
[4,7]
]

我们有个时间轴, 每个时间我们画一条铅笔的总线, 看各个时间的这条总线和横线的区间交点最多是几个

In [12]:
import collections
Event = collections.namedtuple('Event', ('start', 'end'))
Endpoint = collections.namedtuple('Endpoint', ('time', 'is_start'))
def find_max_simultaneous_events(A):    
    E = ([Endpoint(event.start, True) for event in A]) + ([Endpoint(event.end, False) for event in A])
    E.sort(key= lambda e: (e.time, not e.is_start))
    max_num_simutaneous_events, num_simutaneous_events = 0, 0
    for e in E:
        if e.is_start:
            num_simutaneous_events += 1
            max_num_simutaneous_events = max(max_num_simutaneous_events, num_simutaneous_events)
        else:
            num_simutaneous_events -= 1
    return max_num_simutaneous_events

Building Outline

Given N buildings in a x-axis，each building is a rectangle and can be represented by a triple (start, end, height)，where start is the start position on x-axis, end is the end position on x-axis and height is the height of the building. Buildings may overlap if you see them from far away，find the outline of them。

An outline can be represented by a triple, (start, end, height), where start is the start position on x-axis of the outline, end is the end position on x-axis and height is the height of the outline.

Given 3 buildings：

[
  [1, 3, 3],
  [2, 4, 4],
  [5, 6, 1]
]

The outlines are：

[
  [1, 2, 3],
  [2, 4, 4],
  [5, 6, 1]
]


In [19]:
from heapq import heappush, heappop, heapify
import functools
 
class Edge(object):
    def __init__(self, x, height, is_start):
        self.x = x
        self.height = height
        self.is_start = is_start
 
    def __repr__(self):
        return repr((self.x, self.height, self.is_start))
 
 
class Solution(object):
    def compare(self, edge1, edge2):
        if edge1.x != edge2.x:
            return -1 if edge1.x < edge2.x else 1
 
        if ((edge1.is_start and edge2.is_start) or
         (not edge1.is_start and not edge2.is_start)):
            return -1 if edge1.height < edge2.height else 1
 
        return -1 if edge1.is_start else 1
 
    def getSkyline(self, buildings):
        edges = []
        for building in buildings:
            start_edge = Edge(building[0], building[2], True)
            end_edge = Edge(building[1], building[2], False)
            edges.extend((start_edge, end_edge))
        sorted_edges = sorted(edges, key=functools.cmp_to_key(self.compare))
        print(sorted_edges)
        maxHeap, result = [], []
        for edge in sorted_edges:
            if edge.is_start:
                if not maxHeap or edge.height > - maxHeap[0]:
                    result.append((edge.x, edge.height))
                heappush(maxHeap, - edge.height)
            else:
                maxHeap.remove(- edge.height)
                heapify(maxHeap)
                if not maxHeap:
                    result.append((edge.x, 0))
                elif edge.height > - maxHeap[0]:
                    result.append((edge.x, - maxHeap[0]))
 
        return result

buildings = [ [1, 3, 3], [2, 4, 4], [5, 6, 1] ]
slt = Solution()
result = slt.getSkyline(buildings)
print(result)


buildings = [ [1, 6, 6], [2, 5, 4], [3, 4, 1] ]
slt = Solution()
result = slt.getSkyline(buildings)
print(result)

[(1, 3, True), (2, 4, True), (3, 3, False), (4, 4, False), (5, 1, True), (6, 1, False)]
[(1, 3), (2, 4), (4, 0), (5, 1), (6, 0)]
[(1, 6, True), (2, 4, True), (3, 1, True), (4, 1, False), (5, 4, False), (6, 6, False)]
[(1, 6), (6, 0)]
